In [1]:
#!/usr/bin/env python3
%load_ext autoreload
%autoreload 2

This file illustrates how you might experiment with the HMM interface.
You can paste these commands in at the Python prompt, or execute `test_ic.py` directly.
A notebook interface is nicer than the plain Python prompt, so we provide
a notebook version of this file as `test_ic.ipynb`, which you can open with
`jupyter` or with Visual Studio `code` (run it with the `nlp-class` kernel).

In [2]:
import logging, math, os
from pathlib import Path

In [3]:
import torch
from torch import tensor

In [4]:
from corpus import TaggedCorpus
from eval import model_cross_entropy, write_tagging
from hmm import HiddenMarkovModel
from crf import ConditionalRandomField

Set up logging.

In [5]:
log = logging.getLogger("test_ic")       # For usage, see findsim.py in earlier assignment.
logging.root.setLevel(level=logging.INFO)
logging.basicConfig(level=logging.INFO)  # could change INFO to DEBUG
# torch.autograd.set_detect_anomaly(True)    # uncomment to improve error messages from .backward(), but slows down

Switch working directory to the directory where the data live.  You may want to edit this line.

In [6]:
os.chdir("../data")

Get vocabulary and tagset from a supervised corpus.

In [7]:
icsup = TaggedCorpus(Path("icsup"), add_oov=False)
log.info(f"Ice cream vocabulary: {list(icsup.vocab)}")
log.info(f"Ice cream tagset: {list(icsup.tagset)}")

INFO:corpus:Read 40 tokens from icsup
INFO:corpus:Created 4 tag types
INFO:corpus:Created 5 word types
INFO:test_ic:Ice cream vocabulary: ['1', '2', '3', '_EOS_WORD_', '_BOS_WORD_']
INFO:test_ic:Ice cream tagset: ['C', 'H', '_EOS_TAG_', '_BOS_TAG_']


Two ways to look at the corpus ...

In [8]:
os.system("cat icsup")   # call the shell to look at the file directly

1

In [9]:
log.info(icsup)          # print the TaggedCorpus python object we constructed from it

INFO:test_ic:1/C 1/C 1/C 1/C 1/C 1/C 1/C 2/C 2/C 3/H
1/H 2/H 2/H 3/H 3/H 3/H 3/H 3/H 3/H 3/C
1/C 1/C 1/C 1/C 1/C 1/C 1/C 2/C 2/C 3/H
1/H 2/H 2/H 3/H 3/H 3/H 3/H 3/H 3/H 3/C


Make an HMM.

In [10]:
log.info("*** Hidden Markov Model (HMM) test\n")
hmm = HiddenMarkovModel(icsup.tagset, icsup.vocab)
# Change the transition/emission initial probabilities to match the ice cream spreadsheet,
# and test your implementation of the Viterbi algorithm.  Note that the spreadsheet 
# uses transposed versions of these matrices.
hmm.B = tensor([[0.7000, 0.2000, 0.1000],    # emission probabilities
                [0.1000, 0.2000, 0.7000],
                [0.0000, 0.0000, 0.0000],
                [0.0000, 0.0000, 0.0000]])
hmm.A = tensor([[0.8000, 0.1000, 0.1000, 0.0000],   # transition probabilities
                [0.1000, 0.8000, 0.1000, 0.0000],
                [0.0000, 0.0000, 0.0000, 0.0000],
                [0.5000, 0.5000, 0.0000, 0.0000]])
log.info("*** Current A, B matrices (using initalizations from the ice cream spreadsheet)")
hmm.printAB()

INFO:test_ic:*** Hidden Markov Model (HMM) test

INFO:test_ic:*** Current A, B matrices (using initalizations from the ice cream spreadsheet)


Transition matrix A:
	C	H	_EOS_TAG_	_BOS_TAG_
C	0.800	0.100	0.100	0.000
H	0.100	0.800	0.100	0.000
_EOS_TAG_	0.000	0.000	0.000	0.000
_BOS_TAG_	0.500	0.500	0.000	0.000

Emission matrix B:
	1	2	3
C	0.700	0.200	0.100
H	0.100	0.200	0.700
_EOS_TAG_	0.000	0.000	0.000
_BOS_TAG_	0.000	0.000	0.000




Try it out on the raw data from the spreadsheet, available in `icraw``.

In [11]:
log.info("*** Viterbi results on icraw with hard coded parameters")
icraw = TaggedCorpus(Path("icraw"), tagset=icsup.tagset, vocab=icsup.vocab)
write_tagging(hmm, icraw, Path("icraw_hmm.output"))  # calls hmm.viterbi_tagging on each sentence
os.system("cat icraw_hmm.output")   # print the file we just created

INFO:test_ic:*** Viterbi results on icraw with hard coded parameters
100%|██████████| 1/1 [00:00<00:00, 76.79it/s]


1

Did the parameters that we guessed above get the "correct" answer, 
as revealed in `icdev`?

In [12]:
icdev = TaggedCorpus(Path("icdev"), tagset=icsup.tagset, vocab=icsup.vocab)
log.info(f"*** Compare to icdev corpus:\n{icdev}")
from eval import viterbi_error_rate
viterbi_error_rate(hmm, icdev, show_cross_entropy=False)

INFO:test_ic:*** Compare to icdev corpus:
2/H 3/H 3/H 2/H 3/H 2/H 3/H 2/H 2/H 3/H 1/C 3/C 3/C 1/C 1/C 1/C 2/C 1/C 1/C 1/C 3/C 1/C 2/C 1/C 1/C 1/C 2/H 3/H 3/H 2/H 3/H 2/H 2/H
100%|██████████| 1/1 [00:00<00:00, 100.00it/s]
INFO:eval:Tagging accuracy: all: 90.909%, seen: 90.909%, novel: nan%


0.09090909090909094

Now let's try your training code, running it on supervised data.
To test this, we'll restart from a random initialization.
(You could also try creating this new model with `unigram=true`, 
which will affect the rest of the notebook.)

In [13]:
hmm = HiddenMarkovModel(icsup.tagset, icsup.vocab)
log.info("*** A, B matrices as randomly initialized close to uniform")
hmm.printAB()

INFO:test_ic:*** A, B matrices as randomly initialized close to uniform


Transition matrix A:
	C	H	_EOS_TAG_	_BOS_TAG_
C	0.334	0.334	0.332	0.000
H	0.334	0.332	0.334	0.000
_EOS_TAG_	0.334	0.333	0.333	0.000
_BOS_TAG_	0.333	0.334	0.334	0.000

Emission matrix B:
	1	2	3
C	0.333	0.335	0.332
H	0.333	0.333	0.334
_EOS_TAG_	0.000	0.000	0.000
_BOS_TAG_	0.000	0.000	0.000




In [14]:
log.info("*** Supervised HMM training on icsup")
cross_entropy_loss = lambda model: model_cross_entropy(model, icsup)
hmm.train(corpus=icsup, loss=cross_entropy_loss, tolerance=0.0001)
log.info("*** A, B matrices after training on icsup (should "
         "match initial params on spreadsheet [transposed])")
hmm.printAB()

INFO:test_ic:*** Supervised HMM training on icsup
100%|██████████| 4/4 [00:00<00:00, 499.96it/s]
INFO:eval:Cross-entropy: 2.0979 nats (= perplexity 8.149)
100%|██████████| 4/4 [00:00<00:00, 500.26it/s]
INFO:eval:Cross-entropy: 1.3729 nats (= perplexity 3.947)
100%|██████████| 4/4 [00:00<00:00, 500.14it/s]
INFO:eval:Cross-entropy: 1.3729 nats (= perplexity 3.947)
INFO:hmm:Saved model to my_hmm.pkl
INFO:test_ic:*** A, B matrices after training on icsup (should match initial params on spreadsheet [transposed])


Transition matrix A:
	C	H	_EOS_TAG_	_BOS_TAG_
C	0.800	0.100	0.100	0.000
H	0.100	0.800	0.100	0.000
_EOS_TAG_	0.250	0.250	0.250	0.000
_BOS_TAG_	0.500	0.500	0.000	0.000

Emission matrix B:
	1	2	3
C	0.700	0.200	0.100
H	0.100	0.200	0.700
_EOS_TAG_	0.000	0.000	0.000
_BOS_TAG_	0.000	0.000	0.000




Now that we've reached the spreadsheet's starting guess, let's again tag
the spreadsheet "sentence" (that is, the sequence of ice creams) using the
Viterbi algorithm.

In [15]:
log.info("*** Viterbi results on icraw")
icraw = TaggedCorpus(Path("icraw"), tagset=icsup.tagset, vocab=icsup.vocab)
write_tagging(hmm, icraw, Path("icraw_hmm.output"))  # calls hmm.viterbi_tagging on each sentence
os.system("cat icraw_hmm.output")   # print the file we just created

INFO:test_ic:*** Viterbi results on icraw
100%|██████████| 1/1 [00:00<00:00, 125.02it/s]


1

Next let's use the forward algorithm to see what the model thinks about 
the probability of the spreadsheet "sentence."

In [16]:
log.info("*** Forward algorithm on icraw (should approximately match iteration 0 "
             "on spreadsheet)")
for sentence in icraw:
    prob = math.exp(hmm.logprob(sentence, icraw))
    log.info(f"{prob} = p({sentence})")

INFO:test_ic:*** Forward algorithm on icraw (should approximately match iteration 0 on spreadsheet)
INFO:test_ic:9.12755498101095e-19 = p(2 3 3 2 3 2 3 2 2 3 1 3 3 1 1 1 2 1 1 1 3 1 2 1 1 1 2 3 3 2 3 2 2)


Finally, let's reestimate on the icraw data, as the spreadsheet does.
We'll evaluate as we go along on the *training* perplexity, and stop
when that has more or less converged.

In [17]:
log.info("*** Reestimating on icraw (perplexity should improve on every iteration)")
negative_log_likelihood = lambda model: model_cross_entropy(model, icraw)  # evaluate on icraw itself
hmm.train(corpus=icraw, loss=negative_log_likelihood, tolerance=0.0001)

INFO:test_ic:*** Reestimating on icraw (perplexity should improve on every iteration)
100%|██████████| 1/1 [00:00<00:00, 199.79it/s]
INFO:eval:Cross-entropy: 1.2217 nats (= perplexity 3.393)
100%|██████████| 1/1 [00:00<00:00, 199.86it/s]
INFO:eval:Cross-entropy: 1.0807 nats (= perplexity 2.947)
100%|██████████| 1/1 [00:00<00:00, 250.42it/s]
INFO:eval:Cross-entropy: 1.0576 nats (= perplexity 2.879)
100%|██████████| 1/1 [00:00<00:00, 199.92it/s]
INFO:eval:Cross-entropy: 1.0486 nats (= perplexity 2.854)
100%|██████████| 1/1 [00:00<00:00, 166.57it/s]
INFO:eval:Cross-entropy: 1.0438 nats (= perplexity 2.840)
100%|██████████| 1/1 [00:00<00:00, 200.02it/s]
INFO:eval:Cross-entropy: 1.0414 nats (= perplexity 2.833)
100%|██████████| 1/1 [00:00<00:00, 199.89it/s]
INFO:eval:Cross-entropy: 1.0402 nats (= perplexity 2.830)
100%|██████████| 1/1 [00:00<00:00, 215.21it/s]
INFO:eval:Cross-entropy: 1.0396 nats (= perplexity 2.828)
100%|██████████| 1/1 [00:00<00:00, 250.26it/s]
INFO:eval:Cross-entropy: 1.

In [18]:
log.info("*** A, B matrices after reestimation on icraw "
         "should match final params on spreadsheet [transposed])")
hmm.printAB()

INFO:test_ic:*** A, B matrices after reestimation on icraw should match final params on spreadsheet [transposed])


Transition matrix A:
	C	H	_EOS_TAG_	_BOS_TAG_
C	0.934	0.066	0.000	0.000
H	0.072	0.865	0.063	0.000
_EOS_TAG_	0.250	0.250	0.250	0.000
_BOS_TAG_	0.000	1.000	0.000	0.000

Emission matrix B:
	1	2	3
C	0.641	0.148	0.211
H	0.000	0.534	0.466
_EOS_TAG_	0.000	0.000	0.000
_BOS_TAG_	0.000	0.000	0.000




In [19]:
log.info("*** Viterbi results on icraw after reestimation on icraw")
icraw = TaggedCorpus(Path("icraw"), tagset=icsup.tagset, vocab=icsup.vocab)
write_tagging(hmm, icraw, Path("icraw_hmm.output"))  # calls hmm.viterbi_tagging on each sentence
os.system("cat icraw_hmm.output")   # print the file we just created

INFO:test_ic:*** Viterbi results on icraw after reestimation on icraw
100%|██████████| 1/1 [00:00<00:00, 142.85it/s]


1

Now let's try out a randomly initialized CRF on the ice cream data. Notice how
the initialized A and B matrices now hold non-negative potentials,
rather than probabilities that sum to 1.

In [19]:
log.info("*** Conditional Random Field (CRF) test\n")
crf = ConditionalRandomField(icsup.tagset, icsup.vocab)
log.info("*** Current A, B matrices (potentials from small random parameters)")
crf.printAB()

INFO:test_ic:*** Conditional Random Field (CRF) test

INFO:test_ic:*** Current A, B matrices (potentials from small random parameters)


Transition matrix A:
	C	H	_EOS_TAG_	_BOS_TAG_
C	1.004	0.998	1.004	1.003
H	0.996	0.996	1.004	0.988
_EOS_TAG_	0.997	1.015	1.001	1.013
_BOS_TAG_	0.991	1.003	1.001	1.004

Emission matrix B:
	1	2	3	_EOS_WORD_	_BOS_WORD_
C	1.008	0.981	0.992	0.995	1.013
H	1.013	1.004	1.009	0.983	0.986
_EOS_TAG_	0.998	1.010	1.001	0.998	1.019
_BOS_TAG_	0.988	0.998	0.999	1.003	1.014




Now let's try your training code, running it on supervised data. To test this,
we'll restart from a random initialization. 

Note that the logger reports the CRF's *conditional* cross-entropy, 
log p(tags | words) / n.  This is much lower than the HMM's *joint* 
cross-entropy log p(tags, words) / n, but that doesn't mean the CRF
is worse at tagging.  The CRF is just predicting less information.

In [29]:
log.info("*** Supervised CRF training on icsup")
cross_entropy_loss = lambda model: model_cross_entropy(model, icsup)
crf.train(corpus=icsup, loss=cross_entropy_loss, lr=0.1, tolerance=0.0001)
log.info("*** A, B matrices after training on icsup")
crf.printAB()

INFO:test_ic:*** Supervised CRF training on icsup
100%|██████████| 4/4 [00:00<00:00, 235.31it/s]
INFO:eval:Cross-entropy: 1.2552 nats (= perplexity 3.509)
  0%|          | 0/500 [00:00<?, ?it/s]


RuntimeError: The size of tensor a (5) must match the size of tensor b (3) at non-singleton dimension 1

Let's again tag the spreadsheet "sentence" (that is, the sequence of ice
creams) using the Viterbi algorithm.  The trained CRF might get a different
answer than the trained HMM.  (Try comparing the two icraw_*.output files.)

In [ ]:
log.info("*** Viterbi results on icraw with trained parameters")
icraw = TaggedCorpus(Path("icraw"), tagset=icsup.tagset, vocab=icsup.vocab)
write_tagging(crf, icraw, Path("icraw_crf.output"))  # calls hmm.viterbi_tagging on each sentence
os.system("cat icraw_crf.output")   # print the file we just created